In [3]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\users\ychang\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\ychang\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\ychang\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "c:\users\ychang\appdata\local\programs\python\python36\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "c:\users\ychang\appdata\local\programs\python\python36\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL loa

In [2]:
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
valid = valid.fillna(0)
test = test.fillna(0)

In [3]:
feat_col = ['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']
scaler = MinMaxScaler()
train[feat_col] = scaler.fit_transform(train[feat_col])
test[feat_col] = scaler.transform(test[feat_col])
valid[feat_col] = scaler.transform(valid[feat_col])

In [4]:
x_train, y_train = train[feat_col], train.iloc[:,-8:]
x_val, y_val = valid[feat_col], valid.iloc[:,-8:]
x_test, y_test = test[feat_col], test.iloc[:,-8:]

In [5]:
latent = 3
def transform_data(data, label):
    transformed_data, transformed_label = [], []
    users = set(data.index.get_level_values(0))
    for u in tqdm(users):
        d = data.loc[u]
        for i in tqdm(range(latent, len(d)), leave=False):
            transformed_data.append(d.iloc[i-latent+1:i+1])
            transformed_label.append(label.loc[u].iloc[i])
    return np.stack(transformed_data), np.stack(transformed_label)

In [6]:
x_val, y_val = transform_data(x_val, y_val)
x_test, y_test = transform_data(x_test, y_test)
x_train, y_train = transform_data(x_train, y_train)

In [7]:
pickle.dump((x_train, y_train, x_val, y_val, x_test, y_test), open('transformed2.pickle', 'wb'))

In [2]:
x_train, y_train, x_val, y_val, x_test, y_test = pickle.load( open('transformed2.pickle', 'rb'))

In [3]:
model = Sequential()
#model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:
history = model.fit([x_train], batch_size=20, y=y_train, verbose=True, validation_data=(x_val, y_val), shuffle=True,
          epochs=5)

Train on 307354 samples, validate on 37131 samples
Epoch 1/5


  4040/307354 [..............................] - ETA: 8:18:09 - loss: 2.7509 - acc: 0.15 - ETA: 4:20:40 - loss: 2.9532 - acc: 0.17 - ETA: 3:00:29 - loss: 2.8461 - acc: 0.18 - ETA: 2:20:40 - loss: 2.6799 - acc: 0.21 - ETA: 1:57:01 - loss: 2.5407 - acc: 0.25 - ETA: 1:41:09 - loss: 2.4354 - acc: 0.28 - ETA: 1:29:56 - loss: 2.3873 - acc: 0.30 - ETA: 1:21:24 - loss: 2.2700 - acc: 0.33 - ETA: 1:14:49 - loss: 2.1831 - acc: 0.36 - ETA: 1:09:31 - loss: 2.1365 - acc: 0.37 - ETA: 1:05:06 - loss: 2.0636 - acc: 0.38 - ETA: 1:01:27 - loss: 2.0614 - acc: 0.39 - ETA: 58:21 - loss: 2.0415 - acc: 0.3962 - ETA: 55:37 - loss: 2.0369 - acc: 0.40 - ETA: 53:16 - loss: 1.9773 - acc: 0.42 - ETA: 51:14 - loss: 1.9980 - acc: 0.42 - ETA: 49:32 - loss: 1.9530 - acc: 0.43 - ETA: 47:56 - loss: 1.9293 - acc: 0.44 - ETA: 46:32 - loss: 1.9243 - acc: 0.44 - ETA: 45:15 - loss: 1.9282 - acc: 0.44 - ETA: 44:09 - loss: 1.9300 - acc: 0.44 - ETA: 43:06 - loss: 1.9271 - acc: 0.45 - ETA: 42:09 - loss: 1.9152 - acc: 0.45 - ETA: 

 12200/307354 [>.............................] - ETA: 21:50 - loss: 1.3355 - acc: 0.56 - ETA: 21:49 - loss: 1.3354 - acc: 0.56 - ETA: 21:49 - loss: 1.3344 - acc: 0.56 - ETA: 21:48 - loss: 1.3333 - acc: 0.56 - ETA: 21:48 - loss: 1.3326 - acc: 0.56 - ETA: 21:48 - loss: 1.3328 - acc: 0.56 - ETA: 21:47 - loss: 1.3326 - acc: 0.56 - ETA: 21:47 - loss: 1.3313 - acc: 0.56 - ETA: 21:46 - loss: 1.3312 - acc: 0.56 - ETA: 21:46 - loss: 1.3303 - acc: 0.56 - ETA: 21:45 - loss: 1.3301 - acc: 0.56 - ETA: 21:45 - loss: 1.3290 - acc: 0.56 - ETA: 21:45 - loss: 1.3293 - acc: 0.56 - ETA: 21:44 - loss: 1.3298 - acc: 0.56 - ETA: 21:44 - loss: 1.3290 - acc: 0.56 - ETA: 21:43 - loss: 1.3292 - acc: 0.56 - ETA: 21:43 - loss: 1.3281 - acc: 0.56 - ETA: 21:43 - loss: 1.3283 - acc: 0.56 - ETA: 21:43 - loss: 1.3272 - acc: 0.56 - ETA: 21:42 - loss: 1.3269 - acc: 0.56 - ETA: 21:42 - loss: 1.3267 - acc: 0.56 - ETA: 21:41 - loss: 1.3260 - acc: 0.56 - ETA: 21:41 - loss: 1.3253 - acc: 0.56 - ETA: 21:40 - loss: 1.3256 - acc

 16280/307354 [>.............................] - ETA: 20:55 - loss: 1.2737 - acc: 0.57 - ETA: 20:55 - loss: 1.2733 - acc: 0.57 - ETA: 20:55 - loss: 1.2732 - acc: 0.57 - ETA: 20:55 - loss: 1.2730 - acc: 0.57 - ETA: 20:55 - loss: 1.2736 - acc: 0.57 - ETA: 20:55 - loss: 1.2733 - acc: 0.57 - ETA: 20:54 - loss: 1.2725 - acc: 0.57 - ETA: 20:54 - loss: 1.2718 - acc: 0.57 - ETA: 20:54 - loss: 1.2711 - acc: 0.57 - ETA: 20:54 - loss: 1.2713 - acc: 0.57 - ETA: 20:53 - loss: 1.2706 - acc: 0.57 - ETA: 20:53 - loss: 1.2698 - acc: 0.57 - ETA: 20:53 - loss: 1.2691 - acc: 0.57 - ETA: 20:53 - loss: 1.2691 - acc: 0.57 - ETA: 20:53 - loss: 1.2690 - acc: 0.57 - ETA: 20:52 - loss: 1.2688 - acc: 0.57 - ETA: 20:52 - loss: 1.2685 - acc: 0.57 - ETA: 20:52 - loss: 1.2683 - acc: 0.57 - ETA: 20:52 - loss: 1.2682 - acc: 0.57 - ETA: 20:51 - loss: 1.2679 - acc: 0.57 - ETA: 20:51 - loss: 1.2676 - acc: 0.57 - ETA: 20:51 - loss: 1.2675 - acc: 0.57 - ETA: 20:51 - loss: 1.2671 - acc: 0.57 - ETA: 20:51 - loss: 1.2671 - acc

 20360/307354 [>.............................] - ETA: 20:19 - loss: 1.2349 - acc: 0.58 - ETA: 20:18 - loss: 1.2346 - acc: 0.58 - ETA: 20:18 - loss: 1.2344 - acc: 0.58 - ETA: 20:18 - loss: 1.2342 - acc: 0.58 - ETA: 20:18 - loss: 1.2342 - acc: 0.58 - ETA: 20:18 - loss: 1.2342 - acc: 0.58 - ETA: 20:18 - loss: 1.2341 - acc: 0.58 - ETA: 20:18 - loss: 1.2338 - acc: 0.58 - ETA: 20:18 - loss: 1.2334 - acc: 0.58 - ETA: 20:17 - loss: 1.2334 - acc: 0.58 - ETA: 20:17 - loss: 1.2327 - acc: 0.58 - ETA: 20:17 - loss: 1.2324 - acc: 0.58 - ETA: 20:17 - loss: 1.2328 - acc: 0.58 - ETA: 20:17 - loss: 1.2321 - acc: 0.58 - ETA: 20:17 - loss: 1.2320 - acc: 0.58 - ETA: 20:17 - loss: 1.2314 - acc: 0.58 - ETA: 20:16 - loss: 1.2313 - acc: 0.58 - ETA: 20:16 - loss: 1.2306 - acc: 0.58 - ETA: 20:16 - loss: 1.2307 - acc: 0.58 - ETA: 20:16 - loss: 1.2310 - acc: 0.58 - ETA: 20:16 - loss: 1.2309 - acc: 0.58 - ETA: 20:16 - loss: 1.2309 - acc: 0.58 - ETA: 20:16 - loss: 1.2303 - acc: 0.58 - ETA: 20:15 - loss: 1.2302 - acc

...

298060/307354 [============================>.] - ETA: 56s - loss: 0.9240 - acc: 0.66 - ETA: 55s - loss: 0.9240 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 55s - loss: 0.9239 - acc: 0.66 - ETA: 54s - loss: 0.9239 - acc: 0.66 - ETA: 54s - loss: 0.9239 - acc: 0.66 - ETA: 54s - loss: 0.9238 - acc: 0.66 - ETA: 54s - loss: 0.9238 - acc: 0.66 - ETA: 54s - loss: 0.9238 - acc: 0.66 - ETA: 54s - loss: 0.9238 - acc: 0.66 - ETA: 54s - loss: 0.9238 - acc: 0.66 - ETA: 54s - loss: 0.9237 - acc: 0.66 - ETA: 54s - loss: 0.9237 - acc: 0.66 - ETA: 54s - loss: 0.9237 - acc: 0.66 - ETA: 54s - loss: 0.9237 - acc: 0.66 - ETA: 54s - loss: 0.9237 - acc: 0.66 - E

302360/307354 [============================>.] - ETA: 38s - loss: 0.9220 - acc: 0.66 - ETA: 38s - loss: 0.9220 - acc: 0.66 - ETA: 38s - loss: 0.9220 - acc: 0.66 - ETA: 38s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9219 - acc: 0.66 - ETA: 37s - loss: 0.9218 - acc: 0.66 - ETA: 37s - loss: 0.9218 - acc: 0.66 - ETA: 37s - loss: 0.9218 - acc: 0.66 - ETA: 37s - loss: 0.9218 - acc: 0.66 - ETA: 36s - loss: 0.9218 - acc: 0.66 - ETA: 36s - loss: 0.9218 - acc: 0.66 - ETA: 36s - loss: 0.9218 - acc: 0.66 - ETA: 36s - loss: 0.9218 - acc: 0.66 - ETA: 36s - loss: 0.9218 - acc: 0.66 - ETA: 36s - loss: 0.9218 - acc: 0.66 - ETA: 36s - loss: 0.9217 - acc: 0.66 - ETA: 36s - loss: 0.9218 - acc: 0.66 - E

306660/307354 [============================>.] - ETA: 20s - loss: 0.9198 - acc: 0.66 - ETA: 20s - loss: 0.9198 - acc: 0.66 - ETA: 20s - loss: 0.9198 - acc: 0.66 - ETA: 20s - loss: 0.9198 - acc: 0.66 - ETA: 20s - loss: 0.9198 - acc: 0.66 - ETA: 20s - loss: 0.9198 - acc: 0.66 - ETA: 20s - loss: 0.9197 - acc: 0.66 - ETA: 19s - loss: 0.9197 - acc: 0.66 - ETA: 19s - loss: 0.9197 - acc: 0.66 - ETA: 19s - loss: 0.9197 - acc: 0.66 - ETA: 19s - loss: 0.9197 - acc: 0.66 - ETA: 19s - loss: 0.9197 - acc: 0.66 - ETA: 19s - loss: 0.9196 - acc: 0.66 - ETA: 19s - loss: 0.9197 - acc: 0.66 - ETA: 19s - loss: 0.9197 - acc: 0.66 - ETA: 19s - loss: 0.9196 - acc: 0.66 - ETA: 19s - loss: 0.9196 - acc: 0.66 - ETA: 19s - loss: 0.9196 - acc: 0.66 - ETA: 19s - loss: 0.9196 - acc: 0.66 - ETA: 18s - loss: 0.9196 - acc: 0.66 - ETA: 18s - loss: 0.9196 - acc: 0.66 - ETA: 18s - loss: 0.9196 - acc: 0.66 - ETA: 18s - loss: 0.9196 - acc: 0.66 - ETA: 18s - loss: 0.9195 - acc: 0.66 - ETA: 18s - loss: 0.9195 - acc: 0.66 - E

307354/307354 [==============================] - ETA: 2s - loss: 0.9180 - acc: 0.669 - ETA: 2s - loss: 0.9179 - acc: 0.669 - ETA: 2s - loss: 0.9180 - acc: 0.669 - ETA: 2s - loss: 0.9179 - acc: 0.669 - ETA: 2s - loss: 0.9179 - acc: 0.669 - ETA: 2s - loss: 0.9179 - acc: 0.669 - ETA: 2s - loss: 0.9179 - acc: 0.669 - ETA: 2s - loss: 0.9179 - acc: 0.669 - ETA: 2s - loss: 0.9179 - acc: 0.669 - ETA: 2s - loss: 0.9179 - acc: 0.669 - ETA: 1s - loss: 0.9179 - acc: 0.669 - ETA: 1s - loss: 0.9179 - acc: 0.669 - ETA: 1s - loss: 0.9179 - acc: 0.669 - ETA: 1s - loss: 0.9179 - acc: 0.669 - ETA: 1s - loss: 0.9179 - acc: 0.669 - ETA: 1s - loss: 0.9178 - acc: 0.669 - ETA: 1s - loss: 0.9178 - acc: 0.669 - ETA: 1s - loss: 0.9178 - acc: 0.669 - ETA: 1s - loss: 0.9178 - acc: 0.669 - ETA: 1s - loss: 0.9178 - acc: 0.669 - ETA: 1s - loss: 0.9178 - acc: 0.669 - ETA: 1s - loss: 0.9177 - acc: 0.669 - ETA: 0s - loss: 0.9177 - acc: 0.669 - ETA: 0s - loss: 0.9177 - acc: 0.669 - ETA: 0s - loss: 0.9177 - acc: 0.669 - E

KeyboardInterrupt: 

In [5]:
predicted = model.predict(x_test)
predicted = np.argmax(predicted, axis=1)
y_test_decode = np.argmax(y_test, axis=1)
print(accuracy_score(y_test_decode, predicted))

0.41883663290596984


In [6]:
# evaluate the model
_, acc_train = model.evaluate(x_train, y_train, verbose=0)
_, acc_test = model.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (acc_train, acc_test))
# learning curves of model accuracy
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show()

Train: 0.727, Test: 0.419


NameError: name 'history' is not defined